# Probability

> Crypto | 988 | 7 Solves
> 
> Author: Neobeo
> 
> I've been learning about probability distributions, but it's all very confusing so I'm just going to assume that my variant of blackjack gives an advantage to the house. I'll even bet a flag on it.
> 
> nc fun.chall.seetf.sg 30001
> 
> Attachment: [crypto_probability.zip](attachments/crypto_probability.zip)  
> MD5: dd434228be35b701d160121a6504af69

First blooded a bit more than 2 hours into the contest.

Play floating point blackjack against the house using python `random.random()`. Though the game uses floating point numbers which discard some bits of random state each iteration, [it is still possible to obtain the mersenne twister state using this floating point data](https://github.com/qxxxb/ctf/tree/master/2021/zh3r0_ctf/real_mersenne). Start with a decent "stand after ~0.5-0.6 score" strategy to prolong rounds early in the game, then afterwards recover the random state and play optimally by picking moves using dynamic programming.

The text UI of this challenge is not very kind, so some smart usage of `r.recvuntil()` and regex helps.

In [1]:
!wget "https://raw.githubusercontent.com/qxxxb/ctf/master/2021/zh3r0_ctf/real_mersenne/solve.py" -O stolen_untwister.py -q

  HTTP/1.1 200 OK
  Connection: keep-alive
  Content-Length: 6234
  Cache-Control: max-age=300
  Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
  Content-Type: text/plain; charset=utf-8
  ETag: "802e603450052319ce5cca4d073de1462fb771d727258f6c9143dd44bbe08403"
  Strict-Transport-Security: max-age=31536000
  X-Content-Type-Options: nosniff
  X-Frame-Options: deny
  X-XSS-Protection: 1; mode=block
  X-GitHub-Request-Id: 9E5C:3622:1ED2:41C93:62A21CF0
  Accept-Ranges: bytes
  Date: Thu, 09 Jun 2022 16:36:32 GMT
  Via: 1.1 varnish
  X-Served-By: cache-qpg1222-QPG
  X-Cache: HIT
  X-Cache-Hits: 1
  X-Timer: S1654792593.667101,VS0,VE305
  Vary: Authorization,Accept-Encoding,Origin
  Access-Control-Allow-Origin: *
  X-Fastly-Request-ID: 261b5b62809ec74b4ce1556256625b3360f7d51b
  Expires: Thu, 09 Jun 2022 16:41:32 GMT
  Source-Age: 0


In [2]:
import re
from pwn import *
from tqdm.auto import tqdm
from stolen_untwister import Untwister, get_a_b_str

[x] Starting local process '/bin/sh'
[+] Starting local process '/bin/sh': pid 2341


In [3]:
r = remote("fun.chall.seetf.sg", 30001)
r.recvuntil(b"Good luck!  \n")

numbers = []

def parse_turn(turn):
  floats = re.findall(r"\[(\d.\d+)\]", turn)
  floats = [float(i) for i in floats]
  player = re.findall(r"\(p1 = (\d.\d+)\)", turn)[-1]
  player = float(player)
  
  return floats, player

while True:
  turn = r.recvuntil(b"? ").decode()
  floats, player = parse_turn(turn)

  numbers.extend(floats)
  if len(numbers) >= 800:
    break
  # keep hitting until player exceeds 0.6 score
  if player > 0.6:
    r.sendline(b"s")
  else:
    r.sendline(b"h")

[x] Opening connection to fun.chall.seetf.sg on port 30001
[x] Opening connection to fun.chall.seetf.sg on port 30001: Trying 34.131.197.225
[+] Opening connection to fun.chall.seetf.sg on port 30001: Done


In [4]:
ut = Untwister()
for i in numbers:
  a, b = get_a_b_str(int(2 ** 53 * i))
  ut.submit(a)
  ut.submit(b)
rnd = ut.get_random()
oracle_data = [rnd.random() for i in range(10000)]
oracle_data[:5]

Solving...
Solved! (in 6.502s)


[0.43067896598203004,
 0.8849627446744969,
 0.4277448462489709,
 0.6935075844466775,
 0.7393106923514381]

In [5]:
# we resume from last round, give the player a card with their current score
dp_data = [player] + oracle_data[len(floats):]

# dp[number drawn] = (score difference, game count, player starts taking from, player stops taking from, will player bust)
dp = [(-9999, -1, -1, -1, -1) for _ in dp_data]
dp[0] = (0, 0, -1, -1, -1)

winturn = None
for idx in range(len(dp)):
  (cur_score, games, _, _, _) = dp[idx]
  
  # Early stopping
  if (cur_score + games) // 2 == 850: 
    winturn = idx
    break
    
  cur = idx
  
  # player forced to draw first card
  player = dp_data[cur]
  cur += 1

  while player < 1:
    # player stands
    
    # enemy draws until wins or busts
    en_cur = cur
    enemy = dp_data[en_cur]
    en_cur += 1
    while enemy < player and enemy <= 1:
      enemy += dp_data[en_cur]
      en_cur += 1
    
    if enemy >= 1 and dp[en_cur][0] < cur_score + 1:
      # player wins
      dp[en_cur] = (cur_score + 1, games + 1, idx, cur - idx, False)
    elif dp[en_cur][0] < cur_score - 1:
      # enemy wins
      dp[en_cur] = (cur_score - 1, games + 1, idx, cur - idx, False)

    # player hits
    player += dp_data[cur]
    cur += 1
  
  # player bust
  if dp[cur][0] < cur_score - 1:
    dp[cur] = (cur_score - 1, games + 1, idx, cur - idx, True)

dp[winturn-5:winturn+1]

[(387, 1309, 4722, 3, True),
 (389, 1309, 4722, 1, False),
 (389, 1309, 4722, 2, False),
 (388, 1310, 4726, 1, False),
 (388, 1310, 4727, 2, True),
 (390, 1310, 4727, 1, False)]

In [6]:
# recover what moves to take from the dp table
backtrack = []
cur = winturn
while cur != -1:
  _, _, back, take, bust = dp[cur]
  if not bust:
    backtrack.append("s")
  for _ in range(take - 1):
    backtrack.append("h")
  cur = back
backtrack = backtrack[::-1]
backtrack[:5]

['s', 's', 's', 's', 'h']

In [7]:
# send all the moves at once
r.send("\n".join(backtrack).encode())
data = r.clean(0.2)
print(data[-500:].decode())

5)
Do you want to hit or stand? You draw a [0.39427085666512307]. (p1 = 0.8021422333016556)
Do you want to hit or stand? Dealer draws a [0.19214036549336933]. (p2 = 0.19214036549336933)
Dealer draws a [0.5454431306767824]. (p2 = 0.7375834961701517)
Dealer draws a [0.28792670182837277]. (p2 = 1.0255101979985244)
Dealer has gone bust. You win!
Score: 800-537
--------------------------------------------------------------------------------
Here is your flag: SEE{1337_card_counting_24ca335ed1cabbcf}

